In [1]:
import pandas as pd
import json

In [20]:
df_plants = pd.read_excel('../flow_based_domain/plants.xlsx', index_col=0)
df_powerplants = pd.read_csv('../grid/powerplants.csv')

relevant_countries = ['Germany', 'Luxembourg', 'Austria', 'Czech Republic', 'Slovenia', 'Belgium', 'Croatia', 'Hungary', 'Poland', 'France', 'Slovakia', 'Romania', 'Netherlands']
country_zone_map = {
    'Germany': 'DE', 
    'Luxembourg': 'DE', 
    'Austria': 'AT', 
    'Czech Republic': 'CZ', 
    'Slovenia': 'SI', 
    'Belgium': 'BE',
    'Croatia': 'HR', 
    'Hungary': 'HU', 
    'Poland': 'PL', 
    'France': 'FR', 
    'Slovakia': 'SK', 
    'Romania': 'RO', 
    'Netherlands': 'NL',
}

df_powerplants = df_powerplants[df_powerplants.Country.isin(relevant_countries)]
df_powerplants['zone'] = df_powerplants.apply(lambda row : country_zone_map[row.Country], axis=1)
df_powerplants = df_powerplants[~df_powerplants.DateIn.isna()]

In [21]:
df_powerplants['projectID'] = df_powerplants['projectID'].astype('object')
for index, plant in df_powerplants.iterrows():
    if plant['projectID'] != None:
        project = eval(plant['projectID'])
        if 'ENTSOE' in project:
            df_powerplants.at[index, 'projectID'] = list(project['ENTSOE'])
        else:
            df_powerplants.loc[index, 'projectID'] = None
    else:
        df_powerplants.loc[index, 'projectID'] = None

In [22]:
display(df_powerplants[~df_powerplants.projectID.isna()])
display(df_powerplants[~df_powerplants.projectID.isna()][['zone', 'Name']].groupby('zone').count())

,id,Name,Fueltype,Technology,Set,Country,Capacity,Efficiency,DateIn,DateRetrofit,DateOut,lat,lon,Duration,Volume_Mm3,DamHeight_m,StorageCapacity_MWh,EIC,projectID,zone
0,0,Emsland,Nuclear,Steam Turbine,CHP,Germany,1336.000000,0.330000,1988.0,1988.0,2022.0,52.481878,7.306658,NaN,0.0,0.0,0.0,{'11WD7KKE-1K--KW5'},[11WD7KKE-1K--KW5],DE
1,1,Mellach,Hard Coal,Steam Turbine,CHP,Austria,200.000000,NaN,1986.0,1986.0,2020.0,46.911700,15.488300,NaN,0.0,0.0,0.0,{'14W-WML-KW-----0'},[14W-WML-KW-----0],AT
2,2,Eemshaven,Hard Coal,CCGT,PP,Netherlands,1604.170304,0.580000,2015.0,NaN,2029.0,53.440500,6.861200,NaN,0.0,0.0,0.0,{'49W000000000044-'},[49W000000000044-],NL
3,3,Emile Huchet,Hard Coal,CCGT,PP,France,596.493211,NaN,1958.0,2010.0,2022.0,49.152500,6.698100,NaN,0.0,0.0,0.0,"{'17W100P100P0345B', '17W100P100P0344D'}","[17W100P100P0345B, 17W100P100P0344D]",FR
5,5,Belleville,Nuclear,Steam Turbine,PP,France,2620.000000,NaN,1987.0,NaN,2028.0,47.508790,2.875740,NaN,0.0,0.0,0.0,"{'17W100P100P01865', '17W100P100P01857'}","[17W100P100P01865, 17W100P100P01857]",FR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2678,2679,Industriekraftwerk Pck,Oil,Steam Turbine,PP,Germany,359.000000,0.382473,1972.0,2011.0,NaN,53.098180,14.234540,NaN,0.0,0.0,0.0,{'11WD8SCHW5X----D'},[11WD8SCHW5X----D],DE
2695,2696,Herrenhausen Kwh,Natural Gas,CCGT,PP,Germany,100.000000,0.357500,1975.0,1975.0,NaN,52.420000,9.648611,NaN,0.0,0.0,0.0,{'11WD2HERR5GKW--1'},[11WD2HERR5GKW--1],DE
2696,2697,Hamborn,Hard Coal,CCGT,PP,Germany,385.000000,NaN,1958.0,2003.0,NaN,51.494201,6.728433,NaN,0.0,0.0,0.0,{'11WD7BEEC5XHAKW5'},[11WD7BEEC5XHAKW5],DE
2697,2699,Cycofos Tv,Other,NaN,PP,France,62.000000,NaN,2009.0,2009.0,NaN,43.428010,4.886520,NaN,0.0,0.0,0.0,{'17W100P100P0352E'},[17W100P100P0352E],FR


,Name
zone,
AT,32
BE,19
CZ,3
DE,97
FR,99
HU,11
NL,28
PL,52
RO,19


In [23]:
df_powerplants_w_eic = df_powerplants[~df_powerplants.projectID.isna()]

for index, plant in df_plants.iterrows():
    result = df_powerplants_w_eic[df_powerplants_w_eic.projectID.apply(lambda x: plant['eic'] in x)]
    if len(result) > 0:
        df_plants.loc[index, 'construction_year'] = result.iloc[0]['DateIn']
    else:
        df_plants.loc[index, 'construction_year'] = 0

In [24]:
display(df_plants[~df_plants.construction_year.isna()])

,name,eic,type,zone,installed_capacity,lat,lon,node,construction_year
0,GDK-Mellach,14W-WGM-KW-----J,natural_gas,AT,838.0,46.911700,15.488300,1091,0.0
1,Ottensheim-Wilhering,14W-BOW-KW-----9,hydro_run_of_river,AT,179.0,48.316500,14.151057,1280,1974.0
2,FHKW Linz Mitte,14WLSG-MITTE---U,natural_gas,AT,216.8,48.299219,14.322935,690,1970.0
3,Kraftwerk Theiß,14W-KW-THE-EVN-7,natural_gas,AT,485.0,48.394861,15.710147,1064,2000.0
4,FHKW Linz Süd,14WLSG-1SUED---5,natural_gas,AT,158.0,48.267893,14.346898,690,1993.0
...,...,...,...,...,...,...,...,...,...
636,Petrokemija Kutina,NaN,natural_gas,HR,35.0,45.471290,16.792320,1171,0.0
637,Dubrovnik,NaN,hydro_reservoir,HR,216.0,42.603001,18.235001,187,0.0
638,Fuzine,NaN,hydro_pumped_storage,HR,5.7,45.304985,14.714837,412,0.0
639,He Rijeka,NaN,hydro_run_of_river,HR,36.8,45.334999,14.452000,417,0.0


In [25]:
df_plants.construction_year = df_plants.construction_year.astype(int)
df_plants.to_excel('../flow_based_domain/plants.xlsx')